In [1]:
import boto3, pandas as pd 
from time import sleep

In [2]:
AWS_ACCESS_KEY = "<your-key>"
AWS_SECRET_KEY = "<your-secret>"
AWS_REGION = "<region>"
SCHEMA_NAME = "<database-name>"
# S3_STAGING_DIR = "s3://athenas3query-output2/output/"
S3_STAGING_DIR = "s3://athenas3query-output2/"
S3_BUCKET_NAME = "athenas3query-output2"
S3_OUTPUT_DIRECTORY = ""

In [16]:
athena_client=boto3.client('athena',aws_access_key_id=AWS_ACCESS_KEY,
                              aws_secret_access_key=AWS_SECRET_KEY, 
                              region_name=AWS_REGION)

In [5]:
# By Darshil
Dict = {}
def download_and_load_query_results(
              client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    
    while True:
        try:
            # this function will load the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            # athena_client.get_query_execution(
            #     QueryExecutionId=response["QueryExecutionId"]
            # )
            break
        except Exception as err:
            if 'Query did not finish' in str(err):
                sleep(0.01)
            else:
                raise err
    temp_file_location:str ='athena_query_results.csv'
    print(temp_file_location)
    s3_client=boto3.client('s3',aws_access_key_id=AWS_ACCESS_KEY,
                              aws_secret_access_key=AWS_SECRET_KEY, 
                              region_name=AWS_REGION)
    s3_client.download_file(
            S3_BUCKET_NAME,
            # f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv/",
            f"{query_response['QueryExecutionId']}.csv",
            temp_file_location
        )
    return pd.read_csv(temp_file_location)

# download_and_load_query_results(athena_client,response)

In [14]:
# My new function
Dict = {}
def download_query_results(
              client: boto3.client, query: str
) -> pd.DataFrame:
    
    response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={ 
        "OutputLocation": S3_STAGING_DIR, 
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }) 

    while True:
        res=athena_client.get_query_execution(
            QueryExecutionId=response["QueryExecutionId"]
        )
        status=res['QueryExecution']['Status']['State']
        print(status)
        if status=='SUCCEEDED':
            break
        else:
            sleep(0.5)

    print('Query_executed SUCCEEDED')
    temp_file_location:str ='athena_query_results.csv'
    print(temp_file_location)
    s3_client=boto3.client('s3',aws_access_key_id=AWS_ACCESS_KEY,
                              aws_secret_access_key=AWS_SECRET_KEY, 
                              region_name=AWS_REGION)
    print('dowloading the csv')
    s3_client.download_file(
            S3_BUCKET_NAME,
            # f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv/",
            f"{response['QueryExecutionId']}.csv",
            temp_file_location
        )
    return pd.read_csv(temp_file_location)

In [17]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_nytimes_data_in_usa",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={ 
        "OutputLocation": S3_STAGING_DIR, 
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}
    }
) 

In [19]:
response

{'QueryExecutionId': '84322b1e-83e0-491f-b30b-6d69ae2be085',
 'ResponseMetadata': {'RequestId': '6dd8c4de-7a45-43f4-bd0f-1268bf7e87e4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 12 May 2023 06:16:45 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6dd8c4de-7a45-43f4-bd0f-1268bf7e87e4'},
  'RetryAttempts': 0}}

In [23]:
query="SELECT * FROM enigma_jhud limit 5000"
enigma_jhud=download_query_results(client=athena_client,query=query)

RUNNING
SUCCEEDED
Query_executed SUCCEEDED
athena_query_results.csv
dowloading the csv


In [ ]:
enigma_jhud.head(2)

In [24]:
query="SELECT * FROM enigma_nytimes_data_in_usa"
nytimes_data_in_usa=download_query_results(client=athena_client,query=query)

RUNNING
SUCCEEDED
Query_executed SUCCEEDED
athena_query_results.csv
dowloading the csv


In [25]:
query="SELECT * FROM countrycode"
countrycode=download_query_results(client=athena_client,query=query)

query="SELECT * FROM countypopulation"
countypopulation=download_query_results(client=athena_client,query=query)

query="SELECT * FROM countypopulation"
countypopulation=download_query_results(client=athena_client,query=query)

query="SELECT * FROM rearc_covid_19_testing_data"
rearc_covid_19_testing_data=download_query_results(client=athena_client,query=query)

RUNNING
SUCCEEDED
Query_executed SUCCEEDED
athena_query_results.csv
dowloading the csv
RUNNING
SUCCEEDED
Query_executed SUCCEEDED
athena_query_results.csv
dowloading the csv
RUNNING
SUCCEEDED
Query_executed SUCCEEDED
athena_query_results.csv
dowloading the csv
QUEUED
RUNNING
RUNNING
RUNNING
SUCCEEDED
Query_executed SUCCEEDED
athena_query_results.csv
dowloading the csv


In [ ]:
query="SELECT * FROM rearc_usa_hospital_beds limit 2000"
rearc_usa_hospital_beds=download_query_results(client=athena_client,query=query)

In [28]:
query="SELECT * FROM state_abv"
state_abv=download_query_results(client=athena_client,query=query)

RUNNING
SUCCEEDED
Query_executed SUCCEEDED
athena_query_results.csv
dowloading the csv


In [29]:
state_abv.head(2)
new_header=state_abv.iloc[0]
state_abv=state_abv[1:]
state_abv.columns=new_header

In [30]:
state_abv.head(2)

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK


In [31]:
factCovid_1 = enigma_jhud [['fips', 'province_state', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']] 
factCovid_2 =rearc_covid_19_testing_data [['fips', 'date', 'positive', 'negative' ,'hospitalizedcurrently','hospitalized','hospitalizeddischarged' ]]
factCovid=pd.merge(factCovid_1, factCovid_2, on="fips", how="inner")

In [44]:
factCovid.to_csv('factCovid.csv')

In [32]:
factCovid.shape

(1995878, 13)

In [33]:
dimRegion_1 = enigma_jhud[['fips', 'province_state','country_region','latitude','longitude']]
dimRegion_2 = nytimes_data_in_usa[['fips','county','state']]
dimRegion = pd.merge(dimRegion_1, dimRegion_2, on='fips', how='inner')

In [46]:
dimRegion[:5000].to_csv('dimRegion.csv')

In [34]:
dimDate=rearc_covid_19_testing_data[['fips','date']]
dimDate.head()

,fips,date
0,NaN,1061101
1,NaN,20210307
2,NaN,20210306
3,NaN,20210305
4,NaN,20210304


In [35]:
dimDate.head(2)

,fips,date
0,NaN,1061101
1,NaN,20210307


In [36]:
dimDate['date']=pd.to_datetime(dimDate['date']).dt.date
dimDate.head(2)

C:\Users\WASIAH~1\AppData\Local\Temp/ipykernel_26004/3325463671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date']=pd.to_datetime(dimDate['date']).dt.date


,fips,date
0,NaN,1970-01-01
1,NaN,1970-01-01


In [37]:
dimDate['year']=pd.to_datetime(dimDate['date']).dt.year
dimDate['month']=pd.to_datetime(dimDate['date']).dt.month
dimDate['day_of_week']=pd.to_datetime(dimDate['date']).dt.dayofweek
dimDate.head(2)

C:\Users\WASIAH~1\AppData\Local\Temp/ipykernel_26004/3809313168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year']=pd.to_datetime(dimDate['date']).dt.year
C:\Users\WASIAH~1\AppData\Local\Temp/ipykernel_26004/3809313168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month']=pd.to_datetime(dimDate['date']).dt.month
C:\Users\WASIAH~1\AppData\Local\Temp/ipykernel_26004/3809313168.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,fips,date,year,month,day_of_week
0,NaN,1970-01-01,1970,1,3
1,NaN,1970-01-01,1970,1,3


In [38]:
from io import StringIO

In [39]:
csv_buffer=StringIO()

In [40]:
factCovid.to_csv(csv_buffer)

In [42]:
S3_BUCKET_NAME

'athenas3query-output2'

In [ ]:
s3_resource = boto3.resource('s3')
s3_resource.Object(S3_BUCKET_NAME,'factCovid.csv').put(Body=csv_buffer.getvalue())

In [ ]:
csv_buffer.getvalue()